In [1]:
import sys
code_path = '../../network_bio_toolkit'
sys.path.append(code_path)
import create_graph
reload(create_graph)
import heat_and_cluster
reload(heat_and_cluster)
import copy

import community # pip install python-louvain
import networkx as nx
import visJS2jupyter.visJS_module as visJS_module
import visJS2jupyter.visualizations as visualizations
import matplotlib as mpl
import random
import operator
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [90]:
def bias_position_by_partition(pos,partition,r=1.0):
    '''
    Bias the positions by partition membership, to group them together
    
    '''
    
    partition = pd.Series(partition)
    
    for p in list(np.unique(partition)):

        focal_nodes = partition[partition==p].index.tolist()
        for n in focal_nodes:
            pos[n][0]+=pol2cart(r,float(p)/(np.max(partition)+1)*2*np.pi)[0]
            pos[n][1]+=pol2cart(r,float(p)/(np.max(partition)+1)*2*np.pi)[1]
            
    return pos

def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(rho, phi)

def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return(x, y)

def assign_colors_to_clusters(node_to_cluster, cluster_size_cut_off = 20, color_list = None):

    node_to_color = {}
    
    if color_list == None:
        color_list = ['#ff0044', '#a300cc', '#3000b3', '#000c59', '#00d9ca', 
        '#004033', '#00f281', '#00660e', '#88ff00', '#a66f00', '#593000', '#cc5200', '#f22000', '#ff408c', '#731d3f', '#571a66', '#1d3f73', '#40bfff', '#21330d', '#e5c339', '#d96cd2', '#9173e6', '#aacc66', '#736f39', '#403120', '#e5a173', '#e57373', '#d9a3ce', '#40303f', '#acb4e6', '#7c92a6', '#bff2ff', '#b6f2d6', '#d9c7a3', '#806060',
        '#a60016', '#330022', '#004759', '#00735c', '#59000c']
   
    cluster_to_nodes = invert_dict(node_to_cluster)
    
    cluster_to_color = {}
    color_index = 0
    for cluster_id, node_list in cluster_to_nodes.iteritems():
        if len(node_list) < cluster_size_cut_off:
            cluster_to_color[cluster_id] = 'grey'
        else:
            cluster_to_color[cluster_id] = color_list[color_index]
            if color_index < len(color_list)-1:
                color_index = color_index + 1

    for node, cluster_id in node_to_cluster.iteritems():
        node_to_color[node] = cluster_to_color[cluster_id]
    
    return node_to_color

def invert_dict(old_dict):
    inv_dict = {}
    for k, v in old_dict.iteritems():
        inv_dict [v] = inv_dict.get(v, [])
        inv_dict [v].append(k)
    return inv_dict 

## Load Databases

In [2]:
# differentially expressed genes
DEG_filename = "../../DEG_databases/DE_CoeffspaceFlight - groundControl_glds48_20180312.csv"  
DEG_list, DEG_to_pvalue, DEG_to_updown = create_graph.create_DEG_list(DEG_filename, 
                                                                      p_value_filter=0.1,
                                                                      gene_type='symbol',
                                                                      sep = ',')

print "Number of DEG's: " + str(len(DEG_list))

Number of DEG's: 394


In [53]:
# background network
filename = "../../background_networks/10090.protein.actions.v10.5.txt"
G_DEG, DG_universe = create_graph.load_STRING_to_digraph(filename, DEG_list, confidence_filter = 400, gene_type = 'symbol', species = 'mouse')

print "\nNumber of interactions: " + str(len(list(G_DEG.edges())))

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-7303...done.
Finished.
57 input query terms found dup hits:
	[(u'ENSMUSP00000042658', 2), (u'ENSMUSP00000126737', 2), (u'ENSMUSP00000051968', 3), (u'ENSMUSP00000
154 input query terms found no hit:
	[u'ENSMUSP00000036849', u'ENSMUSP00000097354', u'ENSMUSP00000097358', u'ENSMUSP00000137522', u'ENSMU
Pass "returnall=True" to return complete lists of duplicate or missing query terms.

Number of interactions: 2564


## Run Heat Prop

In [56]:
G_heat = nx.Graph(G_DEG)
seed_nodes = [n for n in DEG_list if n in G_DEG]
Wprime = visualizations.normalized_adj_matrix(G_DEG)
visualizations.draw_heat_prop(G_heat,
              seed_nodes,
              Wprime = Wprime,
              num_nodes = 200,
              edge_width=2,
              edge_smooth_enabled=True,
              edge_smooth_type='bezier',
              node_size_multiplier=5,
              hover = False,
              hover_connected_edges = False,
              largest_connected_component = True,
              physics_enabled=True,
              node_font_size=40)  

## Clustering 

In [76]:
# find hottest genes
Fnew = visualizations.network_propagation(G,Wprime,seed_nodes,alpha=.5, num_its=20)
top_genes = Fnew.sort_values(ascending=False)[0:500].index
G_top_genes = DG_universe.subgraph(top_genes)

In [78]:
# cluster hottest genes
G_top_genes = nx.Graph(G_top_genes)
node_to_cluster = community.best_partition(G_top_genes)

In [88]:
# position based on cluster
pos = nx.spring_layout(G_top_genes)
bias_position_by_partition(pos,node_to_cluster,r=1.0); # modifies pos in place

In [91]:
# color based on cluster
node_to_color = assign_colors_to_clusters(node_to_cluster, cluster_size_cut_off = 20, color_list = None)

In [18]:
reload(heat_and_cluster)
heat_and_cluster.draw_clustering(G_DEG,
              DEG_list,
              cluster_size_cut_off = 0,
              Wprime = Wprime,
              num_nodes = 200,
              edge_smooth_enabled=True,
              edge_smooth_type='bezier',
              edge_width=2,
              node_size_multiplier=5,
              hover = False,
              hover_connected_edges = False,
              physics_enabled = True,
              node_font_size=40,
              graph_id = 111)

In [20]:
reload(heat_and_cluster)
html, cluster_pos, rad_pos, node_to_cluster = heat_and_cluster.draw_rad_clustering(G_DEG,
              DEG_list,
              node_spacing = 1000,
              r = 1,
              cluster_size_cut_off = 0,
              Wprime = Wprime,
              num_nodes = 200,
              edge_width=1,
              node_size_multiplier=5,
              hover = False,
              hover_connected_edges = False,
              node_font_size=25,
              graph_id = 11)
html

In [9]:
x_edges = [(1,2),(1,3),(1,4),(1,5),(2,3),(2,4),(2,5),(3,4),(3,5),(4,5),(6,7),(6,8),(6,9),(6,10),(7,8),(7,9),(7,10),(8,9),
            (8,10),(9,10),(4,10),(1,9),(8,3),(6,5),(7,2),(11,2),(11,7),(11,10),(12,3),(12,8),(13,1),(13,10),(15,2),(15,3),(15,13)]
x_G = nx.Graph(x_edges)

In [10]:
x_node_to_cluster = community.best_partition(x_G)

In [11]:
x_node_to_cluster

{1: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 1,
 7: 1,
 8: 1,
 9: 1,
 10: 1,
 11: 1,
 12: 0,
 13: 0,
 15: 0}

In [12]:
x_pos = nx.spring_layout(x_G)
x_pos

{1: array([ 0.46103064,  0.54995217]),
 2: array([ 0.36617802,  0.68440125]),
 3: array([ 0.28746095,  0.29549474]),
 4: array([ 0.25148389,  0.4892981 ]),
 5: array([ 0.50094765,  0.32507271]),
 6: array([ 0.91182003,  0.41316342]),
 7: array([ 0.80632666,  0.73429647]),
 8: array([ 0.72077933,  0.32295785]),
 9: array([ 0.91954019,  0.58398888]),
 10: array([ 0.6271457 ,  0.68946312]),
 11: array([ 0.61486197,  1.        ]),
 12: array([ 0.45971638,  0.        ]),
 13: array([ 0.20174121,  0.86262025]),
 15: array([ 0.        ,  0.60201047])}

In [13]:
x_rad_pos = heat_and_cluster.bias_position_by_partition(x_pos, x_node_to_cluster, r=0.5)
x_rad_pos

{1: array([ 0.96103064,  0.54995217]),
 2: array([ 0.86617802,  0.68440125]),
 3: array([ 0.78746095,  0.29549474]),
 4: array([ 0.75148389,  0.4892981 ]),
 5: array([ 1.00094765,  0.32507271]),
 6: array([ 0.41182003,  0.41316342]),
 7: array([ 0.30632666,  0.73429647]),
 8: array([ 0.22077933,  0.32295785]),
 9: array([ 0.41954019,  0.58398888]),
 10: array([ 0.1271457 ,  0.68946312]),
 11: array([ 0.11486197,  1.        ]),
 12: array([ 0.95971638,  0.        ]),
 13: array([ 0.70174121,  0.86262025]),
 15: array([ 0.5       ,  0.60201047])}

In [21]:
nodes = x_G.nodes()
edges = x_G.edges()
node_to_shape = {n:'dot' if x_node_to_cluster[n] == 0 else 'star' for n in nodes}

# per node attributes
nodes_dict = [{"id":n,
               "color":'pink',
               "node_shape": node_to_shape[n],
               "node_size": 20,
               "x":x_pos[n][0]*700,
               "y":x_pos[n][1]*700} for n in nodes
              ]

# map to indices for source/target in edges
node_map = dict(zip(nodes,range(len(nodes)))) 

# per edge attributes
edges_dict = [{"source":node_map[edges[i][0]], "target":node_map[edges[i][1]], 
              "color":'orange'} for i in range(len(edges))]

# set some network-wide styles
visJS_module.visjs_network(nodes_dict,edges_dict, graph_id = 5473, node_size_multiplier = 10)

In [23]:
# per node attributes
nodes_dict = [{"id":n,
               "color":'pink',
               "node_shape": node_to_shape[n],
               "node_size": 20,
               "x":x_rad_pos[n][0]*2000,
               "y":x_rad_pos[n][1]*2000} for n in nodes
              ]

# set some network-wide styles
visJS_module.visjs_network(nodes_dict,edges_dict, graph_id = 573, node_size_multiplier = 30)

In [25]:
reload(heat_and_cluster)
html, cluster_pos, rad_pos, node_to_cluster = heat_and_cluster.draw_rad_clustering(x_G,
              [1,3,5,7,9],
              node_spacing = 3000,
              r = 20,
              cluster_size_cut_off = 0,
              num_nodes = 300,
              edge_width=2,
              node_size_multiplier=15,
              hover = False,
              hover_connected_edges = False,
              node_font_size= 40,
              graph_id = 490823)
html

In [17]:
reload(heat_and_cluster)
html, cluster_pos, rad_pos, node_to_cluster = heat_and_cluster.draw_rad_clustering(x_G,
              [1,3,5,7,9],
              node_spacing = 3000,
              r = 2,
              cluster_size_cut_off = 0,
              num_nodes = 300,
              edge_width=2,
              node_size_multiplier=5,
              hover = False,
              hover_connected_edges = False,
              physics_enabled = True,
              node_font_size= 40,
              graph_id = 15437)
html